# 01 Import Libraries

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import os

# 02 Import Data

In [3]:
# Create path
path = r'/Users/dana/Documents/Instacart Basket Analysis/'

In [5]:
# import data
df_customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

# 03 Wrangle Data

In [4]:
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
# rename columns
df_customers.rename(columns={'First Name': 'first_name', 'Surnam': 'last_name', 'Gender': 'gender', 'STATE':'state', 'Age': 'age', 'n_dependants': 'number_of_dependants','fam_status':'family_status' }, inplace=True)

In [6]:
df_customers.head()

,user_id,first_name,last_name,gender,state,age,date_joined,number_of_dependants,family_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [7]:
# drop columns
df_customers = df_customers.drop(columns = ['first_name','last_name','gender','state','date_joined'])

In [8]:
df_customers.head()

,user_id,age,number_of_dependants,family_status,income
0,26711,48,3,married,165665
1,33890,36,0,single,59285
2,65803,35,2,married,99568
3,125935,40,0,single,42049
4,130797,26,1,married,40374


In [9]:
df_customers.describe()

,user_id,age,number_of_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [10]:
df_customers.dtypes

user_id                  int64
age                      int64
number_of_dependants     int64
family_status           object
income                   int64
dtype: object

In [11]:
df_customers['family_status'].value_counts()

married                             144906
single                               33962
divorced/widowed                     17640
living with parents and siblings      9701
Name: family_status, dtype: int64

# 04 Data Quality and Consistency Checks

In [12]:
# check for mixed types
for col in df_customers.columns.tolist():
  weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[weird]) > 0:
    print (col)

In [13]:
# check for missing values
df_customers.isnull().sum()

user_id                 0
age                     0
number_of_dependants    0
family_status           0
income                  0
dtype: int64

In [14]:
# check for duplicates
df_dups = df_customers[df_customers.duplicated()]

In [15]:
df_dups

,user_id,age,number_of_dependants,family_status,income


# 05 Combine Data

In [7]:
# import other dataset
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge.pkl'))

In [17]:
# check shapes
ords_prods_merge.shape

(32435070, 22)

In [18]:
df_customers.shape

(206209, 5)

In [23]:
ords_prods_merge.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_last_order', 'new_customer',
       'product_id', 'add_to_cart_order', 'reordered', 'product_name',
       'aisle_id', 'department_id', 'prices', '_merge', 'max_order',
       'loyalty_flag', 'avg_spent', 'spender_flag', 'order_frequency',
       'order_freqency_flag', 'order_frequency_flag'],
      dtype='object')

In [9]:
# remove old _ merge and extra order_frequency_flag
ords_prods_merge = ords_prods_merge.drop(columns = ['_merge','order_frequency_flag'])

In [10]:
# merge data
df_merged = ords_prods_merge.merge(df_customers, on = 'user_id', indicator = True)

In [21]:
df_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,new_customer,product_id,add_to_cart_order,reordered,...,loyalty_flag,avg_spent,spender_flag,order_frequency,order_freqency_flag,order_frequency_flag,age,number_of_dependants,family_status,income
0,2539329.0,1.0,1.0,2.0,8.0,NaN,True,196,1.0,0.0,...,New customer,NaN,NaN,NaN,NaN,NaN,31,3,married,40423
1,2398795.0,1.0,2.0,3.0,7.0,15.0,False,196,1.0,1.0,...,New customer,6.367797,Low spender,20.5,Non-frequent customer,Non-frequent customer,31,3,married,40423
2,473747.0,1.0,3.0,3.0,12.0,21.0,False,196,1.0,1.0,...,New customer,7.515897,Low spender,13.0,Regular customer,Regular customer,31,3,married,40423
3,2254736.0,1.0,4.0,4.0,7.0,29.0,False,196,1.0,1.0,...,New customer,8.197727,Low spender,10.0,Frequent customer,Frequent customer,31,3,married,40423
4,431534.0,1.0,5.0,4.0,15.0,28.0,False,196,1.0,1.0,...,New customer,8.205556,Low spender,20.0,Regular customer,Regular customer,31,3,married,40423


In [11]:
# check shape
df_merged.shape

(32435059, 30)

# 05 Export data

In [12]:
# export dataframe
df_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'df_merged.pkl'))